In [1]:
# use python2

# use only one GPU device
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.system('echo $CUDA_VISIBLE_DEVICES')

0

In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [3]:
config = tf.ConfigProto()

In [4]:
config.gpu_options.allow_growth = True

In [5]:
set_session(tf.Session(config=config))

In [6]:
from keras.models import Model
#from keras.layers import Conv2D, MaxPool2D, Dropout, UpSampling2D, Input, Merge
from keras.layers import Input, Convolution2D, AtrousConvolution2D, MaxPooling2D, merge, ZeroPadding2D, Dropout, UpSampling2D

In [7]:
#from keras.regularizers import l2

In [8]:
#weight_decay = 0.01

In [7]:
#input_shape = (512, 512, 3)
input_shape = (512, 512, 1)
img_input = Input(shape=input_shape)

In [22]:
# building blocks

# block1
h = ZeroPadding2D(padding=(1,1))(img_input)
h = Convolution2D(filters=64, kernel_size=(3,3), activation='relu', name='conv1_1')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=64, kernel_size=(3,3), activation='relu', name='conv1_2')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = MaxPooling2D(pool_size=(3,3), strides=(2,2))(h)

# block2
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=128, kernel_size=(3,3), activation='relu', name='conv2_1')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=128, kernel_size=(3,3), activation='relu', name='conv2_2')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = MaxPooling2D(pool_size=(3,3), strides=(2,2))(h)

# block3
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=256, kernel_size=(3,3), activation='relu', name='conv3_1')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=256, kernel_size=(3,3), activation='relu', name='conv3_2')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=256, kernel_size=(3,3), activation='relu', name='conv3_3')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = MaxPooling2D(pool_size=(3,3), strides=(2,2))(h)

# block4
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=512, kernel_size=(3,3), activation='relu', name='conv4_1')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=512, kernel_size=(3,3), activation='relu', name='conv4_2')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = Convolution2D(filters=512, kernel_size=(3,3), activation='relu', name='conv4_3')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = MaxPooling2D(pool_size=(3,3), strides=(1,1))(h)

# block5 (start to use Atrous convolution)
h = ZeroPadding2D(padding=(2,2))(h)
h = AtrousConvolution2D(filters=512, atrous_rate=(2,2), kernel_size=(3,3), activation='relu', name='conv5_1')(h)
h = ZeroPadding2D(padding=(2,2))(h)
h = AtrousConvolution2D(filters=512, atrous_rate=(2,2), kernel_size=(3,3), activation='relu', name='conv5_2')(h)
h = ZeroPadding2D(padding=(2,2))(h)
h = AtrousConvolution2D(filters=512, atrous_rate=(2,2), kernel_size=(3,3), activation='relu', name='conv5_3')(h)
h = ZeroPadding2D(padding=(1,1))(h)
h = MaxPooling2D(pool_size=(3,3), strides=(1,1))(h)

# atrous (four scales)

# atrous rate = 6
b1 = ZeroPadding2D(padding=(6,6))(h)
b1 = AtrousConvolution2D(filters=1024, atrous_rate=(6,6), kernel_size=(3,3), activation='relu', name='fc6_1')(b1)
b1 = Dropout(0.5)(b1)
b1 = Convolution2D(filters=1024, kernel_size=(1,1), activation='relu', name='fc7_1')(b1)
b1 = Dropout(0.5)(b1)
b1 = Convolution2D(filters=21, kernel_size=(1,1), activation='softmax', name='fc8_1')(b1)

# atrous rate = 12
b2 = ZeroPadding2D(padding=(12,12))(b1)
b2 = AtrousConvolution2D(filters=1024, atrous_rate=(12,12), kernel_size=(3,3), activation='relu', name='fc6_2')(b2)
b2 = Dropout(0.5)(b2)
b2 = Convolution2D(filters=1024, kernel_size=(1,1), activation='relu', name='fc7_2')(b2)
b2 = Dropout(0.5)(b2)
b2 = Convolution2D(filters=21, kernel_size=(1,1), activation='softmax', name='fc8_2')(b2)
                   
# atrous rate = 18
b3 = ZeroPadding2D(padding=(18,18))(b2)
b3 = AtrousConvolution2D(filters=1024, atrous_rate=(18,18), kernel_size=(3,3), activation='relu', name='fc6_3')(b3)
b3 = Dropout(0.5)(b3)
b3 = Convolution2D(filters=1024, kernel_size=(1,1), activation='relu', name='fc7_3')(b3)
b3 = Dropout(0.5)(b3)
b3 = Convolution2D(filters=21, kernel_size=(1,1), activation='softmax', name='fc8_3')(b3)
                   
# atrous rate = 24
b4 = ZeroPadding2D(padding=(24,24))(b3)
b4 = AtrousConvolution2D(filters=1024, atrous_rate=(24,24), kernel_size=(3,3), activation='relu', name='fc6_4')(b4)
b4 = Dropout(0.5)(b4)
b4 = Convolution2D(filters=1024, kernel_size=(1,1), activation='relu', name='fc7_4')(b4)
b4 = Dropout(0.5)(b4)
b4 = Convolution2D(filters=21, kernel_size=(1,1), activation='softmax', name='fc8_4')(b4)
                   
# merge four scales
s = merge([b1, b2, b3, b4], mode='sum')
out = UpSampling2D(size=(8,8))(s)

/root/anaconda2/envs/py2.7_tf1.0/lib/python2.7/site-packages/ipykernel_launcher.py:84: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [24]:
type(img_input)

tensorflow.python.framework.ops.Tensor

In [25]:
type(out)

tensorflow.python.framework.ops.Tensor

In [23]:
# constructing network model
model = Model(inputs=img_input, outputs=out, name="deeplabV2")

In [26]:
type(model)

keras.engine.training.Model

In [27]:
# print model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 1)       0         
_________________________________________________________________
zero_padding2d_59 (ZeroPaddi (None, 514, 514, 1)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 512, 512, 64)      640       
_________________________________________________________________
zero_padding2d_60 (ZeroPaddi (None, 514, 514, 64)      0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 512, 512, 64)      36928     
_________________________________________________________________
zero_padding2d_61 (ZeroPaddi (None, 514, 514, 64)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 256, 256, 64)      0         
__________

In [28]:
# visualize
from keras.utils import plot_model
plot_model(model, to_file='./deeplab_2.png', show_shapes=True, show_layer_names=True)

In [10]:
"""
# block1 
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv1',
          kernel_regularizer=l2(weight_decay))(img_input)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool')(x)

# block2
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block2_pool')(x)

# block3
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block3_pool')(x)

# block4
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block4_pool')(x)

# block5
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block5_pool')(x)

# fully convolutional
x = Conv2D(filters=4096, kernel_size=(7,7), activation='relu', padding='same', name='fc1', 
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)
x = Conv2D(filters=4096,kernel_size=(1,1), activation='relu', padding='same', name='fc2',
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)

# segment
x = Conv2D(filters=21, kernel_size=(1,1), activation='softmax', padding='same',
           kernel_regularizer=l2(weight_decay))(x)

# upsample x 32
x = UpSampling2D(size=(32,32))(x)
"""

In [11]:
# construct final model
model = Model(inputs=img_input, outputs=x)

In [13]:
# print model details (e.g. feature map's size)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0         
__________

In [28]:
??plot_model

In [14]:
# visualize
from keras.utils import plot_model
plot_model(model, to_file='./fcn.png', show_shapes=True, show_layer_names=True)

### fcn.png
https://github.com/amiltonwong/xl_dl/blob/master/keras_seg/fcn.png